# OOP (2024 Fall) HW1: 불변 순서쌍을 활용해 가변 순서쌍 만들기 
- 이름: 배상진
- 학번: 20210551

여기 홍길동, 99999999 대신 본인의 이름, 학번 작성

-----
## Part A: 불변 순서쌍 Pair
1. 클래스(또는 정적) 팩토리 메소드 `of`를 를 수정하여 완성하라.
    - https://velog.io/@cjh8746/%EC%A0%95%EC%A0%81-%ED%8C%A9%ED%86%A0%EB%A6%AC-%EB%A9%94%EC%84%9C%EB%93%9CStatic-Factory-Method
1. 주석 부분을 풀어 제너릭 클래스인 `Pair`가 제너릭 인터페이스 `Comparable`를 구현하도록 완성하라.
    - https://docs.oracle.com/en/java/javase/22/docs/api/java.base/java/lang/Comparable.html

In [3]:
record Pair<F extends Comparable<F>, S extends Comparable<S>>(F first, S second) implements Comparable<Pair<F, S>> /* implements Comparable< ... > */ {
    
    // 아래 팩토리 메소드 of가 컴파일되지 않고 있으니 컴파일되도록 수정하라 (힌트: 제너릭 메소드)
    static <F extends Comparable<F>, S extends Comparable<S>> Pair<F, S> of(F first, S second) {
        return new Pair<>(first, second);
    }
    
    // 사전순이 되도록 정의하기. 즉 first가 더 큰 쪽이 더 크고, first가 같으면 second를 비교
    @Override
    public int compareTo(Pair<F, S> other) {
        int firstComparison = this.first.compareTo(other.first);
        if (firstComparison != 0) {
            return firstComparison;
        }
        return this.second.compareTo(other.second);
    }
}

In [4]:
/* compareTo 가 잘 동작하는지
   - 왼쪽이 더 큰 경우
   - 왼쪽이 더 작은 경우
   - 같은 경우
   이런 경우를 각각 포함하도록 이렇게 최소 3개 이상의 테스트를 실행해 보라
*/

// 첫 번째 Pair: 왼쪽이 더 큰 경우
Pair<Integer, String> pair1 = Pair.of(5, "apple");
Pair<Integer, String> pair2 = Pair.of(3, "banana");
System.out.println(pair1.compareTo(pair2)); // 양수: pair1이 더 큼

// 두 번째 Pair: 왼쪽이 더 작은 경우
Pair<Integer, String> pair3 = Pair.of(2, "apple");
Pair<Integer, String> pair4 = Pair.of(4, "banana");
System.out.println(pair3.compareTo(pair4)); // 음수: pair3이 더 작음

// 세 번째 Pair: first는 같고 second가 다른 경우
Pair<Integer, String> pair5 = Pair.of(3, "apple");
Pair<Integer, String> pair6 = Pair.of(3, "banana");
System.out.println(pair5.compareTo(pair6)); // 음수: second 기준으로 pair5가 더 작음

// 네 번째 Pair: 같은 경우
Pair<Integer, String> pair7 = Pair.of(3, "apple");
Pair<Integer, String> pair8 = Pair.of(3, "apple");
System.out.println(pair7.compareTo(pair8)); // 0: 동일한 값

// 대략 이런 식으로 테스트 실행
// System.out.println( ??.compareTo(??) );
// System.out.println( ??.compareTo(??) );
// System.out.println( ??.compareTo(??) );

1
-1
-1
0


------

In [8]:
class ModifiablePair<F extends Comparable<F>, S extends Comparable<S>> implements Comparable<ModifiablePair<F, S>> /* implements Comparable< ... > */ {
    private Pair<F,S> pair; // 이 field 말고 다른 field는 추가로 정의하지 말 것!

    ModifiablePair(F first, S second) {
        this.pair = Pair.of(first, second);
    }
    
    // getter들은 pair에게 위임(delegate)하는 방식으로 작성하라
    public F first() {
        return pair.first();
    }
    public S second() {
        return pair.second();
    }
    
    // setter들은 적절히 정의하라
    public void setFirst(F first) {
        this.pair = new Pair<>(first, pair.second());
    }
    public void setSecond(S second) {
        this.pair = new Pair<>(pair.first(), second);
    }

    // compareTo는 pair에게 위임(delegate)하는 방식으로 작성하라
    @Override
    public int compareTo(ModifiablePair<F, S> other) {
        return this.pair.compareTo(other.pair);
    }

    @Override
    public String toString() { 
        return pair.toString(); // toString은 그냥 pair에게 위임
    }
}

In [9]:
// ModifiablePair<F,S>의 getter와 setter들이 잘 동작하는지 확인할 수 있는 적절한 테스트 코드 작성하여 실행하라.
// 그러니까 `first()`, `second()`, `setFrist(F)`, `setSecond(S)` 메소드를 모두 활용하는 예시 코드를 작성해 실행하라는 말이다.
// ModifiablePair 객체 생성
        ModifiablePair<Integer, String> modPair = new ModifiablePair<>(1, "apple");

        // 초기값 출력
        System.out.println("Initial state:");
        System.out.println("First: " + modPair.first());  // 기대: 1
        System.out.println("Second: " + modPair.second()); // 기대: "apple"

        // setter를 사용하여 first 값을 변경
        modPair.setFirst(10);
        System.out.println("\nAfter setting first to 10:");
        System.out.println("First: " + modPair.first());  // 기대: 10
        System.out.println("Second: " + modPair.second()); // 기대: "apple"

        // setter를 사용하여 second 값을 변경
        modPair.setSecond("banana");
        System.out.println("\nAfter setting second to banana:");
        System.out.println("First: " + modPair.first());  // 기대: 10
        System.out.println("Second: " + modPair.second()); // 기대: "banana"

        // compareTo 테스트 (10, banana)와 (5, apple) 비교
        ModifiablePair<Integer, String> otherPair = new ModifiablePair<>(5, "apple");
        System.out.println("\nComparison with (5, apple): " + modPair.compareTo(otherPair)); // 기대: 양수

        // compareTo 테스트 (10, banana)와 (10, orange) 비교
        ModifiablePair<Integer, String> anotherPair = new ModifiablePair<>(10, "orange");
        System.out.println("Comparison with (10, orange): " + modPair.compareTo(anotherPair)); // 기대: 음수

        // compareTo 테스트 (10, banana)와 (10, banana) 비교
        ModifiablePair<Integer, String> samePair = new ModifiablePair<>(10, "banana");
        System.out.println("Comparison with (10, banana): " + modPair.compareTo(samePair)); // 기대: 0

Initial state:
First: 1
Second: apple

After setting first to 10:
First: 10
Second: apple

After setting second to banana:
First: 10
Second: banana

Comparison with (5, apple): 1
Comparison with (10, orange): -13
Comparison with (10, banana): 0
